# Title by Owner

## Imports

In [25]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datetime import datetime
from os import path
import tensorflow_addons as tfa
import tensorflow.keras as keras
from utils.callbacks import SaveBestModelInMemory
from utils.submission import create_submission_zip

## Constants

In [26]:
NUM_CLASSES = 12
RANDOM_STATE = 42 # Seed for rng to make everything reproducible and deterministic af
SAVED_MODELS_PATH = "saved-models"
TENSORBOARD_LOGS_PATH = "tensorboard-logs"
SUBMISSIONS_PATH = "../submissions"

## Parameters

In [27]:
BATCH_SIZE = 16 # Number of samples in a mini batch
EPOCHS = 100 # Number of training epochs before the training is stopped
TEST_SPLIT = 0.15 # Percent of data to use for validation/testing

## Data Loading and Preprocessing

In [28]:
data = np.load(file="../dataset/x_train.npy")
labels = tf.keras.utils.to_categorical(np.load(file="../dataset/y_train.npy"), num_classes=NUM_CLASSES)

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=TEST_SPLIT, random_state=RANDOM_STATE)

# Make sure everything was loaded correctly:
print(f"All samples shape: {data.shape}, all labels shape: {labels.shape}")
print(f"Train samples shape: {train_data.shape}, Train labels shape: {train_labels.shape}")
print(f"Test samples shape: {test_data.shape}, Test labels shape: {test_labels.shape}")

All samples shape: (2429, 36, 6), all labels shape: (2429, 12)
Train samples shape: (2064, 36, 6), Train labels shape: (2064, 12)
Test samples shape: (365, 36, 6), Test labels shape: (365, 12)


## Model Definition

In [87]:
from utils.attention import Attention
def build_model(self, input_shape: tuple[int,int,int], nb_classes: int) -> tf.keras.Model:
        n_feature_maps = 64
        input_layer = keras.layers.Input(input_shape)

        # data aug
        data_aug = tf.keras.Sequential([
            tf.keras.layers.GaussianNoise(0.5),
            tf.keras.layers.GaussianDropout(0.5),
        ])(input_layer)
        # conv block -1
        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)
        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)
        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)
        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)
        # conv block -2
        conv2 = keras.layers.Conv1D(filters=64,kernel_size=11,strides=1,padding='same')(output_block_1)
        conv2 = tfa.layers.InstanceNormalization()(conv2)
        conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
        conv2 = keras.layers.Dropout(rate=0.3)(conv2)
        conv2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
        # conv block -3
        conv3 = keras.layers.Conv1D(filters=128,kernel_size=21,strides=1,padding='same')(conv2)
        conv3 = tfa.layers.InstanceNormalization()(conv3)
        conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
        conv3 = keras.layers.Dropout(rate=0.4)(conv3)
        # split for attention
        attention_data = keras.layers.Lambda(lambda x: x[:,:,:64])(conv3)
        attention_softmax = keras.layers.Lambda(lambda x: x[:,:,64:])(conv3)
        # attention mechanism
        attention_softmax = keras.layers.Softmax()(attention_softmax)
        multiply_layer = keras.layers.Multiply()([attention_softmax,attention_data])
        attention_layer = Attention(units=32)(multiply_layer)
        # last layer
        dense_layer = keras.layers.Dense(units=8)(attention_layer)
        act_layer = keras.layers.PReLU()(dense_layer)
        dropout_layer = keras.layers.Dropout(rate=0.4)(act_layer)
        # output layer
        output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(dropout_layer)

        return keras.models.Model(inputs=input_layer, outputs=output_layer)

## Training

In [ ]:
input_shape = train_data.shape[1:]
classes = NUM_CLASSES
model_name = "film-shit" # Give your model an awesome name for a 2% percent accuracy increase.

model = build_model(model_name, input_shape, classes)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam(3e-3),
                      metrics=['accuracy'])
model.summary()

run_id = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
current_tensorboard_log_dir = f"{TENSORBOARD_LOGS_PATH}/{model_name}/{run_id}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=current_tensorboard_log_dir)
print(f"Run tensorboard in a separate process with:\n"
      f"tensorboard --logdir {path.abspath(TENSORBOARD_LOGS_PATH)}\nor\n"
      f"tensorboard --logdir {path.abspath(current_tensorboard_log_dir)}")

best_weights_callback = SaveBestModelInMemory(metric="val_loss")

model.fit(x=train_data, y=train_labels, batch_size=BATCH_SIZE, epochs=450, validation_data=(test_data, test_labels), callbacks=[tensorboard_callback, best_weights_callback])

Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_36 (InputLayer)          [(None, 36, 6)]      0           []                               
                                                                                                  
 conv1d_156 (Conv1D)            (None, 36, 64)       3136        ['input_36[0][0]']               
                                                                                                  
 batch_normalization_76 (BatchN  (None, 36, 64)      256         ['conv1d_156[0][0]']             
 ormalization)                                                                                    
                                                                                                  
 activation_57 (Activation)     (None, 36, 64)       0           ['batch_normalization_76[0

## Optional: Save model in memory

In [29]:
model.set_weights(best_weights_callback.best_weights)
saved_model_path = f"{SAVED_MODELS_PATH}/{model_name}/{run_id}"
model.save(saved_model_path)

INFO:tensorflow:Assets written to: saved-models/My-awesome-model/2022-12-12-19-48-03/assets


INFO:tensorflow:Assets written to: saved-models/My-awesome-model/2022-12-12-19-48-03/assets


## Optional: Create submission ZIP

In [30]:
submission_path = f"{SUBMISSIONS_PATH}/{model_name}/{run_id}"
create_submission_zip(submission_path, saved_model_path)

print(f"Created submission: {submission_path}.zip")


Created submission: ../submissions/My-awesome-model/2022-12-12-19-48-03.zip
